In [43]:
import math
import pandas as pd
import requests
import json
from pyairtable import Api
from pyairtable.formulas import match

In [44]:
#config

project_id = 'TEST_STANDARD'
new_project_id = 'TEST_2021'
new_project_name = 'Projekttest021'

BASE_ID = 'appP3ErdVOT3jS0OM'
API_KEY = 'patmoYYlMYxZQHpSE.c527bb03e732ecdcb79df00166b5b70513e3e29f86f7cf3bcc83922fca713f3f'


In [45]:
api = Api(api_key=API_KEY)

In [46]:
def get_airtable_data(table_name):
    formula = match({"Project": project_id})
    return api.table(base_id=BASE_ID, table_name=table_name).all(formula=formula)

In [47]:
tables = ['Objectives', 'Usecases', 'Models', 'Elements', 'Attributes']

In [48]:
tables = {table_name: get_airtable_data(table_name) for table_name in tables}

In [49]:
# Convert to pandas dataframe and add id column
for table in tables.keys():
    for i in range(len(tables[table])):
        tables[table][i]['fields']['id'] = tables[table][i]['id']
    tables[table] = pd.DataFrame.from_records((r['fields'] for r in tables[table]))

In [50]:
formula = match({"ProjectID": project_id})
df_project = api.table(base_id=BASE_ID, table_name='Projects').all(formula=formula)
df_project = pd.DataFrame.from_records((r['fields'] for r in df_project)).copy()
df_project

,ProjectName,Attribute,ProjectID,ProjectDescriptionEN,Models,OrderVersion,Document,ProjectNumber,Element,Sort,...,ProjectDescriptionDE,Created,CreatedBy,LastModified,UsecaseNameDE,UsecaseCode,ObjectiveEN,ObjectiveDE,LastModifiedBy,RecordID
0,Elementplan Vorlage,"[recLLLaXK9bP81iYb, recfrU4Br4jFNUSna, reclVyn...",TEST_STANDARD,The template project to effectively communicat...,"[reclnkuQvrlOC9aEl, recqXHSPBoi3uRxzQ]",[recosiGFBrE0Pifn9],[receXmvp9N1U02TOo],00000,"[recYvVWmxWRIGoXS4, recJix7UBsz6pIB7y, recDkLF...",0,...,Das Vorlageprojekt um den Standard wirkungsvol...,2022-03-10T10:29:30.000Z,"{'id': 'usreIhaXmobxYfUJe', 'email': 'simon.di...",2023-09-11T15:07:10.000Z,[Managen von Raumbezogenen Attributen im Proje...,[BUC-PUM-3],"[None, None, None, None]",[Verbesserte Kommunikation und Informationsaus...,"{'id': 'usreIhaXmobxYfUJe', 'email': 'simon.di...",rec5E5gc7MaPvDb95


In [51]:
def drop_columns(df, columns_to_drop):
    existing_columns = [col for col in columns_to_drop if col in df.columns]
    if existing_columns:
        df.drop(existing_columns, axis=1, inplace=True)
    return df

columns_to_drop = ['RecordID', 'Project Picture','Created', 'CreatedBy',
       'LastModified','LastModifiedBy','UsecaseNameDE', 'UsecaseNameEN', 'UsecaseNameFR', 'UsecaseNameIT','ObjectiveEN, ObjectiveDE, ObjectiveFR, ObjectiveIT', 'ObjectiveEN','ObjectiveDE', 'UsecaseCode']

df_project = drop_columns(df_project, columns_to_drop)
df_project.columns


Index(['ProjectName', 'Attribute', 'ProjectID', 'ProjectDescriptionEN',
       'Models', 'OrderVersion', 'Document', 'ProjectNumber', 'Element',
       'Sort', 'ProjectDescriptionIT', 'Usecase', 'Objective',
       'ProjectDescriptionFR', 'ProjectDescriptionDE'],
      dtype='object')

In [52]:
# Change the project ID and name
df_project['ProjectID'] = new_project_id
df_project['ProjectName'] = new_project_name
df_project

,ProjectName,Attribute,ProjectID,ProjectDescriptionEN,Models,OrderVersion,Document,ProjectNumber,Element,Sort,ProjectDescriptionIT,Usecase,Objective,ProjectDescriptionFR,ProjectDescriptionDE
0,Projekttest021,"[recLLLaXK9bP81iYb, recfrU4Br4jFNUSna, reclVyn...",TEST_2021,The template project to effectively communicat...,"[reclnkuQvrlOC9aEl, recqXHSPBoi3uRxzQ]",[recosiGFBrE0Pifn9],[receXmvp9N1U02TOo],00000,"[recYvVWmxWRIGoXS4, recJix7UBsz6pIB7y, recDkLF...",0,Il progetto modello per comunicare efficacemen...,"[recC7PojlNVBReAUL, recMvEdu0wgpSrGgy]","[recILSt1AWmzZ0s62, recHnGLjaiVsKKET8, recSgCO...",Le modèle Project pour communiquer efficacemen...,Das Vorlageprojekt um den Standard wirkungsvol...


### Upload empty records to generate new ids

In [53]:
table = api.table(base_id=BASE_ID, table_name='Objectives')
for _ in range(4): table.create({})
table = api.table(base_id=BASE_ID, table_name='Usecases')
for _ in range(2): table.create({})
table = api.table(base_id=BASE_ID, table_name='Models')
for _ in range(2): table.create({})
table = api.table(base_id=BASE_ID, table_name='Elements')
for _ in range(9): table.create({})
table = api.table(base_id=BASE_ID, table_name='Attributes')
for _ in range(48): table.create({})

### Get the new ids

In [54]:
formula = match({'Sort':''})
new_ids_usecases = api.table(base_id=BASE_ID, table_name='Usecases').all(formula=formula)
new_ids_usecases = [r['id'] for r in new_ids_usecases]
new_ids_objectives = api.table(base_id=BASE_ID, table_name='Objectives').all(formula=formula)
new_ids_objectives = [r['id'] for r in new_ids_objectives]
new_ids_models = api.table(base_id=BASE_ID, table_name='Models').all(formula=formula)
new_ids_models = [r['id'] for r in new_ids_models]
new_ids_elements = api.table(base_id=BASE_ID, table_name='Elements').all(formula=formula)
new_ids_elements = [r['id'] for r in new_ids_elements]
new_ids_attributes = api.table(base_id=BASE_ID, table_name='Attributes').all(formula=formula)
new_ids_attributes = [r['id'] for r in new_ids_attributes]

### Get the old ids

In [55]:
usecases_old = tables['Usecases']['id'].to_list()
objectives_old = tables['Objectives']['id'].to_list()
models_old = tables['Models']['id'].to_list()
elements_old = tables['Elements']['id'].to_list()
attributes_old = tables['Attributes']['id'].to_list()

### Update the project links

In [56]:
for u in new_ids_usecases:
    df_project.loc[0,'Usecase'].append(u)
for _ in range(2):
    df_project.loc[0,'Usecase'].pop(0)

for o in new_ids_objectives:
    df_project.loc[0,'Objective'].append(o)
for _ in range(4):
    df_project.loc[0,'Objective'].pop(0)

for m in new_ids_models:
    df_project.loc[0,'Models'].append(m)
for _ in range(2):
    df_project.loc[0,'Models'].pop(0)

for e in new_ids_elements:
    df_project.loc[0,'Element'].append(e)
for _ in range(9):
    df_project.loc[0,'Element'].pop(0)

for a in new_ids_attributes:
    df_project.loc[0,'Attribute'].append(a)
for _ in range(48):
    df_project.loc[0,'Attribute'].pop(0)

df_project

,ProjectName,Attribute,ProjectID,ProjectDescriptionEN,Models,OrderVersion,Document,ProjectNumber,Element,Sort,ProjectDescriptionIT,Usecase,Objective,ProjectDescriptionFR,ProjectDescriptionDE
0,Projekttest021,"[rec2Ma3SblyGNxPyV, rec8z9bYlvq0UQFYo, rec9b7S...",TEST_2021,The template project to effectively communicat...,"[rec2gbTj0UWBXbTCg, recKLCKwCusT6CVKo]",[recosiGFBrE0Pifn9],[receXmvp9N1U02TOo],00000,"[rec1Muj2OptfVVZ4e, recCnlQqAY64E8pKj, recIUX9...",0,Il progetto modello per comunicare efficacemen...,"[rec4r6Kg6lCfbhQ3m, reckPvplCCidkzUPa]","[recEryu16vCbV8YKP, recbPLXU59zsRpfTW, recegdy...",Le modèle Project pour communiquer efficacemen...,Das Vorlageprojekt um den Standard wirkungsvol...


### Upload the new project to airtable

In [57]:
def upload_to_airtable(api_key, base_id, table_name, dataframe):
    
    url = f"https://api.airtable.com/v0/{base_id}/{table_name}"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    responses = []

    # Convert each row in the DataFrame to a dictionary and upload
    for _, row in dataframe.iterrows():
        data = {"fields": row.to_dict()}
        response = requests.post(url, headers=headers, data=json.dumps(data))
        responses.append(response.content)

    return responses

responses = upload_to_airtable(API_KEY, BASE_ID, 'PROJECTS', df_project)
for response in responses:
    print(response)

b'{"id":"recpqwYE1ESwSrtDs","createdTime":"2023-09-20T14:51:54.000Z","fields":{"ProjectID":"TEST_2021","Models":["rec2gbTj0UWBXbTCg","recKLCKwCusT6CVKo"],"Usecase":["rec4r6Kg6lCfbhQ3m","reckPvplCCidkzUPa"],"Element":["rec1Muj2OptfVVZ4e","recCnlQqAY64E8pKj","recIUX9dDTNZ5r6nY","recQibIrk9I3NibWf","recWYtCaIlQu4WeVD","recZ1jhXp4X8hqR2f","reccfqv3TEX6LM850","recjptKGZtROefuOn","recusE9Zh89sMPL4J"],"Attribute":["rec2Ma3SblyGNxPyV","rec8z9bYlvq0UQFYo","rec9b7SqKg2DodWD1","rec9tJ1pbMERXw4oE","recAFZZdigXQ7bDY3","recCinRAVDMzarx8w","recDqKBM2dpbRB4ag","recEwUsJgZqGmZydR","recFKJ5pBydNdon6T","recGTAaxmCxq4Q3py","recH1httyPaefdRcm","recLR0QhU2LSeG8gO","recORIW2ZVkh2HyLO","recTf4eXAznfB5bJz","recVLNtrDZ1L7Rgfp","recWJcY6DtkbPFKOQ","recWLaNfSCBc1gaPQ","recXmol4PjODdIlmI","recZnYrXIyhn5dDln","recbF0Mws8mnHzY8Z","reccv0e2CoR31enAX","recdQINikipUjqVQv","recdV7NMQrND84TmM","recdvE9XGQTFWhjzr","receCnfcTkKqHbpQY","receSDsWPdU0OKjTp","recfViJm9tXIxQa0N","recinKs01EbLhqAPd","recjpGfp4IBKNZEFV","reck8TXX

## Process the tables

In [83]:
new_project_record_id = json.loads(responses[0])['id']

In [59]:
columns_to_drop = ['Project Picture','Created', 'CreatedBy','RecordID', 'ObjectiveEN','Project',
       'LastModified','LastModifiedBy', 'ObjectiveID', 'VersionDate', 'DateMod', 'VersionChanged']
for table in tables.keys():
    tables[table] = drop_columns(tables[table], columns_to_drop)

In [60]:
# Replace usecases ids
for old, new in zip(usecases_old, new_ids_usecases):
    # On the Usecases table
    for i in range(len(tables['Usecases'])):
        if tables['Usecases'].loc[i, 'id'] == old:
            tables['Usecases'].loc[i, 'id'] = new
            break
    # On the Objectives table
    for _, row in tables['Objectives'].iterrows():
        if type(row['Usecase']) == list:
            if old in row['Usecase']:
                row['Usecase'].remove(old)
                row['Usecase'].append(new)
    # On the Models table
    for _, row in tables['Models'].iterrows():
        if type(row['Usecase']) == list:
            if old in row['Usecase']:
                row['Usecase'].remove(old)
                row['Usecase'].append(new)
    # On the Attributes table
    for _, row in tables['Attributes'].iterrows():
        if type(row['Usecase']) == list:
            if old in row['Usecase']:
                row['Usecase'].remove(old)
                row['Usecase'].append(new)
    # On the Elements table
    for _, row in tables['Elements'].iterrows():
        if type(row['Usecase']) == list:
            if old in row['Usecase']:
                row['Usecase'].remove(old)
                row['Usecase'].append(new)

In [61]:
tables['Usecases'].drop(['Models', 'Attribute', 'Elements', 'Objective'], axis=1, inplace=True)

In [62]:
for old, new in zip(attributes_old, new_ids_attributes):
    for i in range(len(tables['Attributes'])):
        if tables['Attributes'].loc[i, 'id'] == old:
            tables['Attributes'].loc[i, 'id'] = new
            break
    for _, row in tables['Elements'].iterrows():
        if type(row['Attribute']) == list:
            if old in row['Attribute']:
                row['Attribute'].remove(old)
                row['Attribute'].append(new)
    for _, row in tables['Models'].iterrows():
        if type(row['Attributes']) == list:
            if old in row['Attributes']:
                row['Attributes'].remove(old)
                row['Attributes'].append(new)

In [63]:
tables['Attributes'].drop(['Element', 'Model'], axis=1, inplace=True)

In [64]:
for old, new in zip(elements_old, new_ids_elements):
    for i in range(len(tables['Elements'])):
        if tables['Elements'].loc[i, 'id'] == old:
            tables['Elements'].loc[i, 'id'] = new
            break
    for _, row in tables['Models'].iterrows():
        if type(row['Element']) == list:
            if old in row['Element']:
                row['Element'].remove(old)
                row['Element'].append(new)

In [65]:
tables['Elements'].drop('Model', axis=1, inplace=True)

In [66]:
for old, new in zip(models_old, new_ids_models):
    for i in range(len(tables['Models'])):
        if tables['Models'].loc[i, 'id'] == old:
            tables['Models'].loc[i, 'id'] = new
            break

for old, new in zip(objectives_old, new_ids_objectives):
    for i in range(len(tables['Objectives'])):
        if tables['Objectives'].loc[i, 'id'] == old:
            tables['Objectives'].loc[i, 'id'] = new
            break

### Update the empty records in airtable

In [67]:
tables['Models'].drop(['ModelNameID', 'Created By', 'Last Modified',
       'Selected(all)', 'Last Modified By', 'VersionNumber', 'ProjectName',
       'Selected', 'Selected(help)'], axis=1, inplace=True)

In [68]:
table = api.table(base_id=BASE_ID, table_name='Models')
for id in new_ids_models:
    dic = tables['Models'][tables['Models']['id'] == id].iloc[0].to_dict()
    dic.pop("id")
    arr = []
    for key in dic.keys():
        # to avoid nan values (float type)
        if type(dic[key]) == float and math.isnan(dic[key]):
            arr.append(key)
    for key in arr:
        dic.pop(key)
    table.update(id, dic)

In [69]:
table = api.table(base_id=BASE_ID, table_name='Objectives')
for id in new_ids_objectives:
    dic = tables['Objectives'][tables['Objectives']['id'] == id].iloc[0].to_dict()
    dic.pop("id")
    arr = []
    for key in dic.keys():
        if type(dic[key]) == float and math.isnan(dic[key]):
            arr.append(key)
    for key in arr:
        dic.pop(key)
    table.update(id, dic)

In [70]:
tables['Elements'].drop(['Selected(all)','Selected(help)','Selected',
        'Created By', 'Last Modified', 'Last Modified By', 'ElementID',
        'Responsible', 'ProjectName', 'Pset', 'Phase', 'ModelNameDE'
        ,'ValuesFR (from Values) (from Attributes)', 'COBieRegisterDE','AttributeNameDE',
        'Direkt Attributes', 'JointtextEN', 'UniquePset', 'JointtextFR','Version',
        'ValuesDE', 'ValuesIT (from Values) (from Attributes)', 'JointtextIT'
        ,'AttributesInPsets','ElementDescriptionIT','PsetsNames', 'IfcEntityName',
        'COBieRegister', 'JointtextDE', 'VersionNumber'], axis=1, inplace=True)

In [71]:
table = api.table(base_id=BASE_ID, table_name='Elements')
for id in new_ids_elements:
    dic = tables['Elements'][tables['Elements']['id'] == id].iloc[0].to_dict()
    dic.pop("id")
    arr = []
    for key in dic.keys():
        if type(dic[key]) == float and math.isnan(dic[key]):
            arr.append(key)
    for key in arr:
        dic.pop(key)
    table.update(id, dic)

In [72]:
tables['Attributes'].drop(['PsetQset','Phase', 'Label',
       'Responsible', 'IfcDataType', 'ValueType',
       'AttributeID', 'ValueDE', 'Created By', 'DescriptionDE', 'ElementNameDE',
       'Last Modified', 'ModelNameDE', 'ElemPsetAttrDE', 'ElemPsetAttrEN',
       'ElemPsetAttrFR', 'ElemPsetAttrIT', 'PsetAttrDE', 'PsetAttrEN',
       'PsetAttrFR', 'PsetAttrIT', 'SortElements', 'UsecaseNameDE',
       'PsetNames', 'Selected(all)', 'Selected(help)',
       'Selected', 'Last Modified By', 'SortElement', 'SortElementMin',
       'SortElementAttribut', 'SortAttributExcel', 'SortElementExcel',
       'ElementDescriptionDE', 'ExcelElementSummaryDE','ValuesIT',
       'ValuesFR', 'ValuesEN', 'IfcEntity', 'IfcEntityName', 'Unit',
       'COBieRegister', 'FachsystemAttribute', 'COBieAttibut', 'ProjectName',
       'COBieAttibutDE', 'COBieRegisterDE', 'DataFieldName', 'FachsystemDE',
       'prob'], axis=1, inplace=True)


In [73]:
table = api.table(base_id=BASE_ID, table_name='Attributes')
for id in new_ids_attributes:
    dic = tables['Attributes'][tables['Attributes']['id'] == id].iloc[0].to_dict()
    dic.pop("id")
    arr = []
    for key in dic.keys():
        if type(dic[key]) == float and math.isnan(dic[key]):
            arr.append(key)
    for key in arr:
        dic.pop(key)
    table.update(id, dic)

In [74]:
tables['Usecases'].drop(['Selected','UsecaseID','FileName',
    'Version', 'Responsible', 'Phase', 'Status', 'ModelNameDE',
    'Created By', 'Last Modified', 'Last Modified By'], axis=1, inplace=True)

In [75]:
table = api.table(base_id=BASE_ID, table_name='Usecases')
for id in new_ids_usecases:
    dic = tables['Usecases'][tables['Usecases']['id'] == id].iloc[0].to_dict()
    dic.pop("id")
    arr = []
    for key in dic.keys():
        if type(dic[key]) == float and math.isnan(dic[key]):
            arr.append(key)
    for key in arr:
        dic.pop(key)
    table.update(id, dic)

-------

# Clean Up

In [79]:
# api = Api(api_key=API_KEY)

In [81]:
# new_project_id

'TEST_2021'

In [85]:
# new_project_record_id

'recpqwYE1ESwSrtDs'

In [80]:
# def delete_project(project_id, new_project_record_id):
#     formula = match({"Project": project_id})
#     table = api.table(base_id=BASE_ID, table_name='Elements')
#     elements = table.all(formula=formula)
#     for element in elements: table.delete(element['id'])

#     table = api.table(base_id=BASE_ID, table_name='Attributes')
#     attributes = table.all(formula=formula)
#     for attribute in attributes: table.delete(attribute['id'])

#     table = api.table(base_id=BASE_ID, table_name='Models')
#     models = table.all(formula=formula)
#     for model in models: table.delete(model['id'])

#     table = api.table(base_id=BASE_ID, table_name='Usecases')
#     usecases = table.all(formula=formula)
#     for usecase in usecases: table.delete(usecase['id'])

#     table = api.table(base_id=BASE_ID, table_name='Objectives')
#     objectives = table.all(formula=formula)
#     for objective in objectives: table.delete(objective['id'])

#     table = api.table(base_id=BASE_ID, table_name='Projects')
#     table.delete(new_project_record_id)

In [87]:
# delete_project(new_project_id, new_project_record_id)